# Render de modelos 3D en Realidad Aumentada con Pygfx

[Pygfx](https://pygfx.org/) es una biblioteca de Python para el render de escenas 3D que implementa [PBR (Physically Based Rendering)](https://en.wikipedia.org/wiki/Physically_based_rendering) y capaz de interpretar modelos en formato [glTF 2.0](https://www.khronos.org/gltf/) propuesto por el grupo [Khronos](https://www.khronos.org/), responsable de múltiples estándares abiertos.

>     pip install pygfx



>     pip install gltflib
>     pip install imageio   #dependencia de gltflib

Pygfx ofrece una API muy simple de usar para la carga y render de modelos, a la vez que presenta un rendimiento y calidad destacados. La composición de la escena en Pygfx la vamos a realizar con las clases escenaPYGFX y modeloGLTF definidos en cuia.py

El renderizado de una escena 3D necesita:
* Uno o varios modelos 3D.
* Una o varias luces, imprescindibles para que se pueda ver el modelo. Una de las luces puede ser una luz ambiental omnipresente que no necesita que se indique su ubicación.
* Una cámara en la que se hará la proyección de la escena.

Modelos, luces y cámara deben estar ubicados en la escena en ubicaciones y posiciones relativas a un origen de coordenadas. En Realidad Aumentada, el origen de coordenadas de la escena lo ubicaremos en el centro del marcador Aruco y la cámara estará en la posición relativa de la webcam con respecto al marcador.

In [3]:
import cv2
import numpy as np
import cuia

Cargamos los parámetros de la cámara (el id lo definimos en la variable cam) desde el fichero generado por el calibrado [camara.py](camara.py). Si no existe entonces definimos unos parámetros de una cámara ideal sin distorsiones.

In [4]:
cam = 0
bk = cuia.bestBackend(cam)

# Obtenemos el alto y ancho de los frames capturados por la cámara
webcam = cv2.VideoCapture(cam,bk)
ancho = int(webcam.get(cv2.CAP_PROP_FRAME_WIDTH))
alto = int(webcam.get(cv2.CAP_PROP_FRAME_HEIGHT))
webcam.release()

try:
    # Importamos la matriz característica de la cámara y sus coeficientes de distorsión del fichero de calibrado
    import camara
    cameraMatrix = camara.cameraMatrix
    distCoeffs = camara.distCoeffs
except ImportError:
    # Si la cámara no estaba calibrada suponemos que no presenta distorsiones
    cameraMatrix = np.array([[ 1000,    0, ancho/2],
                             [    0, 1000,  alto/2],
                             [    0,    0,       1]])
    distCoeffs = np.zeros((5, 1)) 


La localización, rotación y escala de los elementos de la escena 3D se especifica mediante [matrices de transformación](https://es.wikipedia.org/wiki/Matriz_de_transformaci%C3%B3n), que representan dichas operaciones en forma de matrices 4x4. Para la creación y composición de dichas matrices utilizaremos la clase matrizDeTransformacion definida en [cuia.py](cuia.py).

En una escena necesitamos 3 elementos
* Uno o más modelos 3D ubicados en algún punto del espacio con respecto a un eje de coordenadas de la escena.
* Una o más fuentes de iluminación.
* Una cámara que realizará la proyección 2D de la escena.

Hay muchos formatos de representación de modelos 3D. Por ahora solo consideraré el formato [glTF (GL Transmission Format)](https://es.wikipedia.org/wiki/GlTF) propuesto por el grupo [Khronos](https://es.wikipedia.org/wiki/Khronos_Group). Emplearé la clase modeloGLTF definida en [cuia.py](cuia.py).

El objetivo es mezclar la imagen del mundo real que captura la webcam con la imagen del mundo virtual que captura la cámara de la escena. Para que esta mezcla sea correcta es necesario que ambas cámaras tengan las mismas características. En el caso de Pygfx utilizaremos una [PerspectiveCamera](https://docs.pygfx.org/stable/_autosummary/cameras/pygfx.cameras.PerspectiveCamera.html#pygfx.cameras.PerspectiveCamera) que necesita que indiquemos el ancho y alto de la imagen capturada y el [campo de visión (FoV)](https://es.wikipedia.org/wiki/Campo_de_visi%C3%B3n) expresado en grados, de la menos de estas dimensiones, que suele ser la vertical. Para calcular el FoV necesitamos la matriz característica de la cámara que ya tendremos si hicimos previamente el [calibrado](04-OpenCV-Calibrado.ipynb) de la cámara.

In [5]:
modelo = cuia.modeloGLTF('../media/pera.glb')
modelo.rotar((np.pi/2.0, 0, 0)) # Rotar el modelo 90 grados en X para que coincida con el punto de vista que se obtiene en Blender
modelo.escalar(0.15) # Escalado uniforme del modelo
modelo.flotar() # Sitúa el modelo en el rango positivo del eje Z
# Reproducimos las animaciones que haya en el modelo
lista_animaciones = modelo.animaciones()
if len(lista_animaciones) > 0:
    modelo.animar(lista_animaciones[0])

**Nota**: el sistema de coordenadas usado por Pygfx (el mismo que usa OpenGL) tiene una orientación distinta al que usa OpenCV. 

![Sistems de coordenadas de OpenCV y Pygfx](media/opencvpygfx.png "Sistems de coordenadas de OpenCV y Pygfx")

Por ello será necesario hacer la conversión adecuada para el uso combinado de OpenCV y Pygfx.

Dada la pose percibida del marcador y calculada mediante [solvePnP](https://docs.opencv.org/4.x/d9/d0c/group__calib3d.html#ga549c2075fac14829ff4a58bc931c033d), que tendremos especificada en forma de un vector de translación **tvec** y un vector de rotación **rvec**, necesitamos expresarla en forma de matriz de transformación y adaptarla al modelo de sistema de coordenadas de Pygfx. Esto implicará el uso de la función [Rodrigues](https://docs.opencv.org/4.x/d9/d0c/group__calib3d.html#ga61585db663d9da06b68e70cfbf6a1eac) y la inversión de las componentes Y y Z.

In [6]:
def fromOpencvToPygfx(rvec, tvec):
    pose = np.eye(4)
    pose[0:3,3] = tvec.T
    pose[0:3,0:3] = cv2.Rodrigues(rvec)[0]
    pose[1:3] *= -1  # Inversión de los ejes Y y Z
    pose = np.linalg.inv(pose)
    return(pose)

Esta función será la que nos indique la matriz de transformación empleada para ubicar la cámara en Pygfx (en la misma posición relativa de la webcam con respecto al marcador). El resto de elementos de la escena los ubicaremos en una posición relativa al origen de coordenadas ubicado en el centro del marcador.

In [7]:
def fov(cameraMatrix, ancho, alto):
    if ancho > alto:
        f = cameraMatrix[1, 1]
        fov_rad = 2 * np.arctan(alto / (2 * f))
    else:
        f = cameraMatrix[0, 0]
        fov_rad = 2 * np.arctan(ancho / (2 * f))
    return np.rad2deg(fov_rad)

Usaremos la clase escenaPYGFX para crear una nueva escena y añadir el modelo e iluminación.

In [8]:
escena = cuia.escenaPYGFX(fov(cameraMatrix, ancho, alto), ancho, alto)
escena.agregar_modelo(modelo)
escena.ilumina_modelo(modelo)
escena.iluminar() # Agrega iluminación ambiental

La clase escenaPYGFX agrega automáticamente la cámara a la escena. La posición y orientación de la cámara se especificará mediante una [matrizDeTransformacion](cuia.py) que se irá actualizando en función de la pose detectada del marcador de Aruco.

In [9]:
ar = cuia.myVideo(cam, bk) # Iniciamos un objeto myVideo

Para cada frame debemos actualizar la cámara en función de la detección del marcador Aruco. Utilizaremos diccionario 5X5_50

In [10]:
diccionario = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_5X5_50)
detector = cv2.aruco.ArucoDetector(diccionario)

def detectarPose(frame, tam): # tam es el tamaño en metros del marcador
    bboxs, ids, rechazados = detector.detectMarkers(frame)
    print("ids: ",ids)
    if ids is not None:
        print("ha entrado al if")
        objPoints = np.array([[-tam/2.0, tam/2.0, 0.0],
                              [tam/2.0, tam/2.0, 0.0],
                              [tam/2.0, -tam/2.0, 0.0],
                              [-tam/2.0, -tam/2.0, 0.0]])
        resultado = {}
        for i in range(len(ids)):
                ret, rvec, tvec = cv2.solvePnP(objPoints, bboxs[i], cameraMatrix, distCoeffs)
                if ret:
                    resultado[ids[i][0]] = (rvec, tvec)
        return((True, resultado))
    return((False, None))

La función realidadMixta será la que procese cada frame, actualice la cámara, obtenga el render de la escena y combine la imagen obtenida con el frame de la cámara.

In [11]:
def realidadMixta(frame):
    ret, pose = detectarPose(frame, 0.19)
    if ret and pose[0]:
        M = fromOpencvToPygfx(pose[0][0], pose[0][1])
        escena.actualizar_camara(M)
        imagen_render = escena.render()
        imagen_render_bgr = cv2.cvtColor(imagen_render, cv2.COLOR_RGBA2BGRA)
        resultado = cuia.alphaBlending(imagen_render_bgr, frame)
    else:
        resultado = frame
        
    return resultado

En definitiva el proceso que ha de realizarse para cada frame consta de los siguientes pasos:
* Detectar el marcador y obtener la pose (translación y rotación)
* Ubicar la cámara de la escena 3D en la posición de la webcam
* Combinar el renderizado del modelo 3D con la imagen de la webcam

In [12]:
ar.process = realidadMixta
try:
    ar.play("AR", key=ord(' '))
finally:
    ar.release()

[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]
[[0]]


: 

![Prueba de render](media/rendertierra.png "Prueba de render")